In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Zeroshot
- T5
- Flan-T5
- GPT2
- DistilBERT
- RoBERTa
- Llama2 (with PEFT & LoRA)

In [2]:
predmodels_list = {"0" :"T5:t5", "1" : "Flan-T5:flant5", "2":"GPT2:gpt2","3":"DistilBERT:distilbert",
                   "4":"RoBERTa:roberta","5":"Llama2:llama2", "6":"BERTSquad:bert"}

predict_now=True
embed_now=True
compute_metrics_now=True

# Install Necessary Libararies

In [3]:
!pip install -Uq transformers
!pip install -Uq evaluate
!pip install -Uq SentencePiece
!pip install -Uq sentence-transformers
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (set

In [4]:
# !pip install accelerate>=0.20.1
# !pip install transformers[torch]
# # You need to restart the kernel after this step

# Load Libraries and Configurations

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import ast
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, RandomSampler
from torch.nn.functional import cosine_similarity
import torch.nn.functional as F

from sentence_transformers import SentenceTransformer, util

# import tensorflow as tf

# import spacy
# import string

from sklearn.model_selection import train_test_split

import transformers
import evaluate  # Bleu


import nltk
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
# @title Load Configuraiton 
import hbqaconfig as conf

In [7]:
import torch

# # Detect and initialize TPU
# tpu_available = tf.config.experimental.list_logical_devices("TPU")
# if tpu_available:
#     print("TPU available")
# else:
#     print("No TPU available")

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

print(DEVICE)

cuda


# Load QA Dataset

In [8]:
# df = pd.read_csv(datapath + '06-HBQA_Manual_with_Chunk.csv') # for local machine/vedavit colab

# Uncomment below code
# !pip install -q gdown
# import gdown

# # Replace the shared link with the actual link to your file
# file_url = 'https://drive.google.com/uc?id=1Euvnmp8yJ2LGlL2uDvDjER87PYz9RVvS'
# output_path = '/content/hbqa-colab.txt'  # Specify the desired file name and path

# gdown.download(file_url, output_path, quiet=False)

df = pd.read_csv(datapath + '06-HBQA_Manual_with_Chunk.csv')


In [9]:
print(df.shape)
df.head(2)

(1102, 11)


,Ques_Id,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
0,0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


In [10]:
df.set_index('Ques_Id',inplace=True)
df.head(2)

,Chunk_Id,Section_Id,Question,Ref_Answer,Chunk,Reference,WordsInQues,WordsInAns,WordsInRef,WordsInChunk
Ques_Id,,,,,,,,,,
0,389,Book03_002,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,Even this is eternal morality. They that perfo...,The significance of the Agnihotra and the cons...,16,50,50,809
1,390,Book03_003,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"Section III\n""Vaisampayana said, 'Yudhishthira...",Yudhishthira's predicament and his consultatio...,14,41,53,852


In [11]:

# df['WordsInChunk'] = df.Chunk.str.split(' ').apply(len)
# df['CharInChunk'] = df.Chunk.apply(len)
# df.to_csv(r'H:\My Drive\HBQA\Data\06-HBQA_Manual_with_Chunk.csv')

In [12]:
# sample code
# from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# TOKENIZER = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
# MODEL = AutoModelForQuestionAnswering.from_pretrained("hf-internal-testing/tiny-random-ReformerForQuestionAnswering")
# MODEL.to(DEVICE)

# Common Setting for Training

In [13]:
Question_Len = int(max(len(ques) for ques in df.Question)/4)
Answer_Len = int(max(len(ans) for ans in df.Ref_Answer)/4)
Chunk_Len = int(max(len(chunk) for chunk in df.Chunk)/4)
Chunk_Len, Question_Len,Answer_Len # in Tokens

(1499, 51, 326)

In [14]:
Q_LEN =  1500 #256   # Question Length
T_LEN =  500 #32  # Target Length
BATCH_SIZE = 4
# DEVICE = "cuda:0"

# Load Zeroshot Prediction Model

In [15]:
# Zero shot Transformers/Models
def load_model(predmodel_name):
  if predmodel_name=="t5":
    from transformers import T5Tokenizer,  T5ForConditionalGeneration #, T5Model , T5TokenizerFast
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
    model.to(DEVICE)
  elif predmodel_name=="gpt2":
    from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium",  padding_side="left")  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium").to(DEVICE)  #also try gpt2, gpt2-large and gpt2-medium, also gpt2-xl
  elif predmodel_name=="bert":
    from transformers import AutoTokenizer, AutoModelForQuestionAnswering
    model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
    # model_name = "t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

  return tokenizer, model


In [16]:
predmodel_name = predmodels_list['6'].split(':')[1]
print(predmodel_name)
tokenizer, model = load_model(predmodel_name)

bert


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Function to predict Answers

In [17]:
def predict_answer(context, question, tokenizer, model):
    if predmodel_name=="t5":
      inputs = tokenizer(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

      input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
      attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

      outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100)

      predicted_answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)

    elif predmodel_name=="gpt2":
      Q_LEN=1024
      # inputs = tokenizer(context, question, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)
      # input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
      # attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)
      # outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=100)

      # Tokenize the input and set pad_token_id
      input_text = f"{context} [SEP] {question}"
      input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=Q_LEN, truncation=True, padding="max_length", pad_to_max_length=True)

      # Set pad_token_id to eos_token_id
      model.config.pad_token_id = model.config.eos_token_id

      # Generate text
      outputs = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)

      predicted_answer = tokenizer.decode(outputs.flatten(), skip_special_tokens=True)

    elif predmodel_name=="bert":
      inputs = tokenizer(question, context, return_tensors="pt", padding=True, max_length=512, truncation=True)

      with torch.no_grad():

          outputs = model(**inputs)


      start_scores = outputs.start_logits
      end_scores = outputs.end_logits

      answer_start = torch.argmax(start_scores)
      answer_end = torch.argmax(end_scores)+50
      predicted_answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end+1])



    return predicted_answer

# Predict Answers

In [18]:
from IPython.display import clear_output
clear_output()

In [19]:
from IPython.display import display
from IPython.display import HTML

In [20]:
if predict_now:
  df_pred = pd.DataFrame(columns=['Ques_Id','Question','Ref_Answer','Pred_Answer'])

def pred_ans(df_pred, sample=True, verbose=False):
  import random
  if sample:
    qno=random.sample(set(df.index),10)
  else:
    qno = df.index

  total_items = len(qno)
  j=0

  # progress_bar = tqdm(total=total_items, position=0, leave=True)

  progress_bar = tqdm(total=total_items, position=0, leave=True, dynamic_ncols=True)
  progress_html = HTML("""<progress value="0" max="{0}" style="width: 100%"></progress>""".format(total_items))


  for i  in qno:
      progress_bar.update(j)
      progress_html.value = """<progress value="{0}" max="{1}" style="width: 100%"></progress>""".format(i, total_items)
      progress_bar = tqdm(total=total_items, position=0, leave=True)
      ques_id  = df.index[i]
      chunk    = df.iloc[i]['Chunk']
      # print(chunk)
      # chunk = chunk.replace("'","").replace("\","")
      ques     = df.iloc[i]['Question']
      ref_ans  = df.iloc[i]['Ref_Answer']

      pred_ans = predict_answer(chunk, ques, tokenizer, model)

      df_pred.loc[j] = ques_id, ques, ref_ans, pred_ans
      j+=1
      if verbose:
        print('Question  :', ques)
        print("Ref Answer:", ref_ans)
        print("Pred Ans  :", pred_ans)
        print('--------')

      # print(f"Prediting Answer {i}/{df.shape[0]}")

  # progress_bar.close()


In [21]:
# @title Default title text
if predict_now:
  pred_ans(df_pred, sample=False, verbose=False)

  0%|          | 0/1102 [00:00<?, ?it/s]


In [22]:
if predict_now:
  df_pred.to_csv(datapath + '09.11-' + predmodel_name +'Predicted_Ans-Zeroshot.csv', index=False)

df_pred = pd.read_csv(datapath + '09.11-' + predmodel_name +'Predicted_Ans-Zeroshot.csv')
print(df_pred.shape)
df_pred.head(2)


(1102, 4)


,Ques_Id,Question,Ref_Answer,Pred_Answer
0,0,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,sin for such neglect. none should cook his foo...
1,1,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"unable to support them. i cannot abandon them,..."


# Load Embedding Model

In [23]:
#Select Model Function

# https://www.sbert.net/docs/pretrained_models.html

#250MB, multi-qa-distilbert-cos-v1',  Max Sequence Length:	512, Dimensions:768, Normalized Embeddings:	true
#80MB, all-MiniLM-L6-v2, Max Sequence Length:	256, Dimensions:	384, Normalized Embeddings:	true
#290MB, all-distilroberta-v1, Max Sequence Length:	512, Dimensions:	768, Normalized Embeddings:	true
#420MB, all-mpnet-base-v2, Max Sequence Length:	384, Dimensions:	768, Normalized Embeddings:	true
#1.36GB, all-roberta-large-v1, Max Sequence Length:	256, Dimensions: 1024, Normalized Embeddings:	true

def select_embmodel(num):
    emb_modelshortlist = ['distilbert','minilm','distilroberta','mpnet','roberta']

    emb_modellist = ['multi-qa-distilbert-cos-v1',
                'all-MiniLM-L6-v2',
                'all-distilroberta-v1',
                'multi-qa-mpnet-base-dot-v1',
                'all-roberta-large-v1']

    embmodelname = emb_modellist[num]
    embmodelshort = emb_modelshortlist[num]
    embmodelname1 = "_" + embmodelname

    print (embmodelname,'\t',embmodelshort,'\t', embmodelname1)
    return embmodelname, embmodelshort, embmodelname1

In [24]:
embmodelname, embmodelshort, embmodelname1 = select_embmodel(3)

multi-qa-mpnet-base-dot-v1 	 mpnet 	 _multi-qa-mpnet-base-dot-v1


In [25]:
if embed_now:
  embmodel = SentenceTransformer(embmodelname)

# Vectorize Answer

In [26]:
df_pred =  pd.read_csv(datapath + '09.11-' + predmodel_name +'Predicted_Ans-Zeroshot.csv')

df_pred.set_index('Ques_Id',inplace=True)

In [27]:
df_pred.head(3)

,Question,Ref_Answer,Pred_Answer
Ques_Id,,,
0,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,sin for such neglect. none should cook his foo...
1,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"unable to support them. i cannot abandon them,..."
2,What advice does Dhaumya offer to Yudhishthira...,Dhaumya advises Yudhishthira to take refuge in...,all living beings that had been created were s...


In [28]:
def create_ans_vector():

    Ans_Sentences = df_pred.Pred_Answer.tolist()
    Ans_Embeddings = embmodel.encode(Ans_Sentences)

    print ("Answer Embedding: ", Ans_Embeddings.shape)
    # return (Ques_Embeddings, Ans_Embeddings)

    return (Ans_Embeddings)

In [29]:
df_pred['Pred_Answer'] = df_pred.Pred_Answer.fillna("don't know")
df_pred.loc[df_pred['Pred_Answer'].str.contains(r'\?\?\?'), 'Pred_Answer'] = "don't know"

# Model couldn't fine any any answer for this
df_pred.loc[df_pred['Pred_Answer'].str.contains(r"don't know")].shape


(51, 3)

In [30]:
if embed_now:
  Ans_Embeddings = create_ans_vector()
  df_pred['Pred_Answer_Vector'] = torch.tensor(Ans_Embeddings, dtype=torch.float).tolist()

Answer Embedding:  (1102, 768)


In [31]:
if embed_now:
  df_pred.to_csv(datapath + '09.11-' + predmodel_name +'Predicted_AnsVec-Zeroshot.csv')

# df_pred = pd.read_csv(datapath + '09.11-' + predmodel_name +'Predicted_AnsVec-Zeroshot.csv')
print(df_pred.shape)
df_pred.head(2)

(1102, 4)


,Question,Ref_Answer,Pred_Answer,Pred_Answer_Vector
Ques_Id,,,,
0,What is the significance of performing the Agn...,Performing the Agnihotra is considered importa...,sin for such neglect. none should cook his foo...,"[-0.0690019428730011, 0.008900998160243034, -0..."
1,"What predicament does Yudhishthira face, and h...",Yudhishthira faces the predicament of being un...,"unable to support them. i cannot abandon them,...","[-0.12635889649391174, -0.06858988851308823, -..."


# Calculate Metrics

In [32]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics import precision_score, recall_score
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer
import numpy as np

smoother = SmoothingFunction()
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

def get_nlp_metrics(ques_id, ref_ans, pred_ans):

  # Calculate ROUGE-1 and ROUGE-L scores
  scores = scorer.score(ref_ans, pred_ans)

  # Access individual ROUGE scores
  rouge_1_precision = scores['rouge1'].precision
  rouge_1_recall = scores['rouge1'].recall
  rouge_1_f1 = scores['rouge1'].fmeasure

  rouge_l_precision = scores['rougeL'].precision
  rouge_l_recall = scores['rougeL'].recall
  rouge_l_f1 = scores['rougeL'].fmeasure

  rouge_1_precision, rouge_1_recall, rouge_1_f1, rouge_l_precision, rouge_l_recall, rouge_l_f1,

  ref_tokens = word_tokenize(ref_ans)
  pred_tokens = word_tokenize(pred_ans)

  # Calculate BLEU score for a single sentence
  bleu_score = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smoother.method2)

  # Calculate BLEU score for multiple sentences
  # corpus_bleu_score = corpus_bleu([[ref_tokens]], [pred_tokens],  smoothing_function=smoother.method2)

  # print(set(ref_tokens).intersection(set(pred_tokens)))
  tp = len(set(ref_tokens).intersection(set(pred_tokens)))
  precision = tp / len(pred_tokens)
  recall =  tp / len(ref_tokens)

  return ques_id, bleu_score, rouge_1_precision, rouge_1_recall, rouge_1_f1, rouge_l_precision, rouge_l_recall, rouge_l_f1, precision, recall


In [33]:
# from datasets import load_metric
if compute_metrics_now:
  df_metrics = pd.DataFrame(columns = ['Ques_Id','BLEU1', 'ROUGE1_P', 'ROUGE1_R', 'ROUGE1_F1',
                              'ROUGEL_P', 'ROUGEL_R', 'ROUGE1_F1', 'Precision', 'Recall'])
  N= df_pred.shape[0]

  for i in range(N):
    # bleu_score1 = calculate_score( df1.iloc[i]['ref_answer'],df1.iloc[i]['pred_answer'])
    ques_id = df_pred.index[i]
    ref_ans  = df_pred.loc[df_pred.index==ques_id,'Ref_Answer'].values[0]
    pred_ans = df_pred.loc[df_pred.index==ques_id,'Pred_Answer'].values[0]
    results = get_nlp_metrics(ques_id, ref_ans, pred_ans)
    df_metrics.loc[i] = results


In [34]:
metricsfile = '09.12-' + predmodel_name +'Predicted_Ans_Score_Zeroshot.csv'
print(metricsfile)

if compute_metrics_now:
  df_metrics.to_csv(datapath + metricsfile, index=False)

# df_metrics = pd.read_csv(datapath + metricsfile)
print(df_metrics.shape)
df_metrics.head(3)

(1102, 10)


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGE1_F1,Precision,Recall
0,0.0,0.041990,0.309524,0.260000,0.282609,0.190476,0.160000,0.173913,0.239130,0.180328
1,1.0,0.060333,0.210526,0.195122,0.202532,0.157895,0.146341,0.151899,0.166667,0.177778
2,2.0,0.071492,0.367347,0.352941,0.360000,0.183673,0.176471,0.180000,0.258621,0.250000


In [35]:
df_metrics = pd.read_csv(datapath + '09.12-' + predmodel_name +'Predicted_Ans_Score.csv')
print(df_metrics.shape)
df_metrics.head(3)

(1102, 10)


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGE1_F1.1,Precision,Recall
0,0.0,0.041990,0.309524,0.260000,0.282609,0.190476,0.160000,0.173913,0.239130,0.180328
1,1.0,0.060333,0.210526,0.195122,0.202532,0.157895,0.146341,0.151899,0.166667,0.177778
2,2.0,0.071492,0.367347,0.352941,0.360000,0.183673,0.176471,0.180000,0.258621,0.250000


# Calculate Cosine between Predicted and Reference Answer

In [36]:
df_ref_ans = pd.read_csv(datapath + '07.2-HBQA_QA_Vector' + embmodelname1 + '.csv')

df_ref_pred= df_pred.merge(df_ref_ans, on = "Ques_Id", how="inner")[['Ques_Id','Pred_Answer_Vector','AnsVector_multi-qa-mpnet-base-dot-v1']]

In [37]:
df_ref_pred

,Ques_Id,Pred_Answer_Vector,AnsVector_multi-qa-mpnet-base-dot-v1
0,0,"[-0.0690019428730011, 0.008900998160243034, -0...","[-0.316677063703537, 0.04548855870962143, -0.2..."
1,1,"[-0.12635889649391174, -0.06858988851308823, -...","[-0.013333199545741081, 0.05390685796737671, -..."
2,2,"[0.24459676444530487, -0.420696884393692, -0.2...","[-0.06047853082418442, -0.12258763611316681, -..."
3,3,"[0.09395932406187057, -0.2535439133644104, -0....","[-0.5039571523666382, -0.3825620412826538, -0...."
4,4,"[-0.035163771361112595, -0.17514553666114807, ...","[0.1670873612165451, -0.22499610483646393, -0...."
...,...,...,...
1097,1097,"[-0.1877906322479248, -0.431467741727829, -0.4...","[-0.13338713347911835, -0.15367954969406128, -..."
1098,1098,"[0.21185094118118286, -0.146537646651268, -0.2...","[-0.002032045042142272, -0.09146470576524734, ..."
1099,1099,"[0.13243292272090912, -0.34101778268814087, -0...","[0.02510783076286316, -0.6113841533660889, -0...."
1100,1100,"[0.0056121028028428555, -0.1891413927078247, -...","[0.31003451347351074, -0.05984937772154808, -0..."


In [38]:
def calculate_cosine(row):

    if type(row['Pred_Answer_Vector'])!=type(list()):
      predAns_vector_values = ast.literal_eval(row['Pred_Answer_Vector'])
    else:
      predAns_vector_values = row['Pred_Answer_Vector']

    if type(row['AnsVector' + embmodelname1])!=type(list()):
      refAns_vector_values = ast.literal_eval(row['AnsVector' + embmodelname1])
    else:
      refAns_vector_values = row['AnsVector' + embmodelname1]


    # Convert the string values to floats
    predAns_vector_values = [float(value) for value in predAns_vector_values]
    refAns_vector_values = [float(value) for value in refAns_vector_values]

    # Convert the lists to PyTorch tensors
    predAns_vector_values = torch.tensor(predAns_vector_values).reshape(1, -1)
    refAns_vector_values = torch.tensor(refAns_vector_values).reshape(1, -1)

    similarity = cosine_similarity(predAns_vector_values, refAns_vector_values)
    return similarity.item()


In [39]:
df_ref_pred['Cosine'] = df_ref_pred.apply(calculate_cosine, axis=1)


In [40]:
df_ref_pred.head(2)

,Ques_Id,Pred_Answer_Vector,AnsVector_multi-qa-mpnet-base-dot-v1,Cosine
0,0,"[-0.0690019428730011, 0.008900998160243034, -0...","[-0.316677063703537, 0.04548855870962143, -0.2...",0.581825
1,1,"[-0.12635889649391174, -0.06858988851308823, -...","[-0.013333199545741081, 0.05390685796737671, -...",0.478114


In [41]:
df_metrics = df_metrics.merge(df_ref_pred, on="Ques_Id", how="left")[['Ques_Id', 'BLEU1', 'ROUGE1_P', 'ROUGE1_R', 'ROUGE1_F1', 'ROUGEL_P',
       'ROUGEL_R', 'ROUGE1_F1.1', 'Precision', 'Recall','Cosine']]

# Save File Metrics & Show Summary

In [42]:
metricsfile = '09.12-' + predmodel_name +'Predicted_Ans_Score_Zeroshot.csv'
print(metricsfile)
df_metrics.to_csv(datapath + metricsfile, index=False)
df_metrics.head(2)

09.12-bertPredicted_Ans_Score_Zeroshot.csv


,Ques_Id,BLEU1,ROUGE1_P,ROUGE1_R,ROUGE1_F1,ROUGEL_P,ROUGEL_R,ROUGE1_F1.1,Precision,Recall,Cosine
0,0.0,0.041990,0.309524,0.260000,0.282609,0.190476,0.160000,0.173913,0.239130,0.180328,0.581825
1,1.0,0.060333,0.210526,0.195122,0.202532,0.157895,0.146341,0.151899,0.166667,0.177778,0.478114


In [43]:
df_metrics.mean()

Ques_Id        550.500000
BLEU1            0.046807
ROUGE1_P         0.236422
ROUGE1_R         0.288613
ROUGE1_F1        0.224632
ROUGEL_P         0.164100
ROUGEL_R         0.200024
ROUGE1_F1.1      0.154663
Precision        0.152517
Recall           0.187705
Cosine           0.536720
dtype: float64